<a href="https://colab.research.google.com/github/andersonnb6/AdressFinder-Online/blob/main/AdressFinder_Online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AdressFinder-Online**  
Constitui-se de um buscador de endereços desenvolvido em linguagem R.  
Este buscador se baseia na consulta online do Código de Endereçamento Postal (CEP) ao banco de dados diponível no site https://cepaberto.com/.

### Requesitos para o uso

*   Credenciamento no site https://cepaberto.com/.
*   Gerar token de acesso ao banco de dados.
*   Ter uma lista de CEPs em formato .csv ([exemplo](https://www.exemplo.com)).



### Definindo token de acesso banco de dados
*Adicione o token a variável abaixo.*

In [ ]:
token_key = 'a8c99e5231f62500e671cd64a8617825'

### Limpando o ambiente e definindo opções

In [19]:
rm(list=ls()) # limpando ambiente
options(scipen=999) # desabilitando notação científica
options(warn = -1) # ignorando warnings

### Carregando pacotes

In [20]:
# Package names
packages <- c("dplyr", "cepR", "RCurl")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))

### Carregando funções

In [21]:
sleep_sec <- function(x) { # Gerar uma espera de tempo
  p1 <- proc.time()
  Sys.sleep(x)
  proc.time() - p1 
}

### Carregando banco de dados

In [ ]:
# Carregando o banco de dados
download <- getURL("https://raw.githubusercontent.com/andersonnb6/RScripts/main/CEP_finder/consulta_online/input_file.csv")
df <- read.csv(text = download)
df

### Executando busca e gerando resultados

In [ ]:
# Listas onde serão armazenados os resultados
line <- c()
identifier <- c()
state <- c()
city <- c()
district <- c()
public_area <- c()
latitude <- c()
longitude <- c()

# Loop buscador de CEP
for (i in 1:nrow(df)) {

  # consulta no site https://cepaberto.com/
  adress <- busca_cep(cep=paste(df$CEP[i]), token = token_key)
  
  # Espera de 1.3 segundos até a próxima consulta
  sleep_sec(1.3)
  
  # Armazenando resultados da consulta
  identifier <- append(df$IDENTIFICADOR[i],identifier)
  state <-  append(adress$estado,state)
  city <-  append(adress$cidade,city)
  district <- append(adress$bairro,district)
  public_area <- append(adress$logradouro, public_area)
  latitude <- append(adress$latitude, latitude)
  longitude <- append(adress$longitude, longitude)
  line <- append(i, line)
  
  # Mostrando resultado (etapa) obtida no momento
  # (Descomente a linha abaixo para acompanhar o andamento quando estiver executando fora do COLAB)
  ##print(paste(line[1], identifier[1], state[1], city[1], district[1], latitude[1], longitude[1], sep = "    "))
}

# Criando tabela final
final_table <- data.frame(line, identifier, state, city, district, public_area, latitude, longitude)

# Mostrando a tabela
print(final_table)

# Salvando resultados tabulados
write.csv(final_table, file = "output_file.csv",sep = ",",quote = FALSE,row.names = FALSE, col.names = FALSE)